In [ ]:
import torch
import torch.autograd as autograd

def grad(y, x):
    g = autograd.grad(y, [x], grad_outputs=torch.ones_like(y), create_graph=True)[0]
    return g

def div(y, x):
    div = 0.0
    for i in range(y.shape[-1]):
        div += autograd.grad(y[..., i], x, grad_outputs=torch.ones_like(y[..., i]), create_graph=True)[0][..., i:i+1]
    return div

def Laplacian(y, x):
    g = grad(y, x)
    return div(g, x)

In [ ]:
def f(x):
  return x[0,0]**2 + x[0,1]**2 - 1 

In [ ]:
x = torch.tensor([[2.0,2.0]], requires_grad=True)
y = f(x)
z = Laplacian(y, x)
print(y)
print(z)

tensor(7., grad_fn=<SubBackward0>)
tensor([[4.]], grad_fn=<AddBackward0>)


In [ ]:
# From https://github.com/tranbaquang1708/GraduationThesis/blob/a255b86348590985e4f2be9cc1f47f7a62b38ecd/modules/Operation.py
#------------------------------------------------
# Compute grad
def compute_grad(inputs, outputs):
  # var = torch.autograd.Variable(points, requires_grad=True).to(points.device)
  # outputs = model(var)
  g = torch.autograd.grad(outputs=outputs,
                          inputs=inputs, 
                          grad_outputs=torch.ones_like(outputs, requires_grad=False, device=outputs.device), 
                          create_graph=True,
                          retain_graph=True, 
                          only_inputs=True)[0][:, -3:]

  return g

def compute_laplacian(model, inputs, p=2):
  g = compute_grad(inputs, model(inputs))
  # g = g.abs().pow(p-2) * g

  div = 0.
  for i in range(g.shape[-1]):
    div += torch.autograd.grad(g[..., i], inputs, grad_outputs=torch.ones_like(g[..., i]), create_graph=True)[0][..., i:i+1]
    # div += (p-1) * g[..., i].abs().pow(p-2).view(g.shape[0],1) * d

  return div

In [ ]:
x = torch.tensor([[2.0,2.0]], requires_grad=True)
z = compute_laplacian(f, x, 2)
print(z)

tensor([[4.]], grad_fn=<AddBackward0>)
